In [1]:
data_path="C:\\Users\\ruttu\\Desktop\\Mask Detector\\dataset"

In [2]:
import os
import cv2

In [3]:
categories=["with_mask","without_mask"]
map_target={"with_mask":0,"without_mask":1}

# Data Fetching

In [4]:
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(data_path,category)
    images=os.listdir(folder_path)
    
    print(f'[INFO] loading images of category {category}...')
    
    for image in images:
        img=os.path.join(folder_path,image)
        try:
            img=cv2.imread(img)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized_img=cv2.resize(img,(100,100))
            data.append(resized_img)
            target.append(map_target[category])
        except:
            pass
print("Fetching data completed........")

[INFO] loading images of category with_mask...
[INFO] loading images of category without_mask...
Fetching data completed........


In [5]:
import numpy as np

In [6]:
data=np.array(data)
#Converting the colored images to greyscale 
data=data/255


In [7]:
data.shape

(3810, 100, 100)

In [8]:
#Althogh when we converted to grayscale the imgs still have 3 color channels(BGR),so we have to RESHAPE it. 
data = np.reshape(data, (3810, 100, 100, 1))

In [9]:
data.shape

(3810, 100, 100, 1)

In [10]:
target=np.array(target)

In [11]:
target.shape

(3810,)

In [12]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
new_target=to_categorical(target)

In [14]:
new_target.shape

(3810, 2)

# Defining Architecture

In [15]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D

In [16]:
model=Sequential()


model.add(Conv2D(200,(3,3),input_shape=(100,100,1),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(200,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(200,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(200,(3,3),activation="relu"))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(2,activation="softmax"))

# Compiling the model

In [17]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

# Fit the model or Train the model

In [18]:
model.fit(data,new_target,epochs=20)

Epoch 1/20
3810/3810 [==============================] - 313s 82ms/step - loss: 0.6069 - accuracy: 0.6533
Epoch 2/20
3810/3810 [==============================] - 310s 81ms/step - loss: 0.3936 - accuracy: 0.8231
Epoch 3/20
3810/3810 [==============================] - 310s 81ms/step - loss: 0.3110 - accuracy: 0.8675
Epoch 4/20
3810/3810 [==============================] - 309s 81ms/step - loss: 0.2679 - accuracy: 0.8864
Epoch 5/20
3810/3810 [==============================] - 312s 82ms/step - loss: 0.2382 - accuracy: 0.9052
Epoch 6/20
3810/3810 [==============================] - 313s 82ms/step - loss: 0.2111 - accuracy: 0.9155
Epoch 7/20
3810/3810 [==============================] - 425s 112ms/step - loss: 0.1841 - accuracy: 0.9270
Epoch 8/20
3810/3810 [==============================] - 605s 159ms/step - loss: 0.1654 - accuracy: 0.9386
Epoch 9/20
3810/3810 [==============================] - 605s 159ms/step - loss: 0.1523 - accuracy: 0.9404
Epoch 10/20
3810/3810 [=============================

# Face Detection

In [19]:
cascPath = os.path.dirname(cv2.__file__) + "\data\haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

In [22]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,180)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
    

font=cv2.FONT_HERSHEY_SIMPLEX
categories=np.array(categories)

while True:
    success,result=cap.read()
    if not success:
        print("Can't receive the result.EXITING.........")
        break
        
        
    grayimg=cv2.cvtColor(result,cv2.COLOR_BGR2GRAY)
    img=np.asarray(result)
    img=cv2.resize(img,(100,100))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img/255
    
    
    faces=faceCascade.detectMultiScale(
    grayimg,
    scaleFactor=1.1,
    minNeighbors=4,
    minSize=(30,30),
    flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    
    cv2.imshow("Processed Image",img)
    img=np.reshape(img,(1,100,100,1))
    
    #Making our model to predict
    predictions=model.predict(img)
    category_index=model.predict_classes(img)
    prob=np.amax(predictions)
    
    
    if(prob>0.7):
        
        #to make the rectangles around our faces
        for (x,y,w,h) in faces:
            
            if(categories[category_index][0]=="with_mask"):
                color=(0,200,0)
            else:
                color=(0,0,200)
                
            cv2.rectangle(result,(x,y),(x+w,y+h),color,3)
            cv2.rectangle(result,(x,y-40),(x+w,y),color,-1)
            cv2.putText(result,str(round((prob*100),2))+"%",(x+10,y-10),font,0.75,(255,255,255),2)
            cv2.rectangle(result,(x,y+h+3),(x+w,y+h+40),color,-1)
            cv2.putText(result,str(categories[category_index][0]),(x+10,y+h+45),font,0.75,(255,255,255),2)
    cv2.imshow("Result",result)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
model.save("my_mask_model.h5")